# 🎨 ai-gen - LoRA Training & Image Generation

**Train LoRAs and generate ultra-realistic images with NO restrictions**

---

## 🚀 Quick Start

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Run Cell 1**: Setup (installs everything)
3. **Access GUI**: Click the ngrok URL when it appears

---

In [ ]:
# CELL 1: Complete Setup & Launch
# Run this cell - takes about 10-15 minutes

print("🚀 Starting ai-gen setup...\n")

# Download and run setup script
!wget -q https://raw.githubusercontent.com/SamuelD27/ai-gen/main/colab_setup.py
!python colab_setup.py